## Problem Statement
You are working as a data scientist in a global finance company. Over the years, the company has collected basic bank details and gathered a lot of credit-related information. The management wants to build an intelligent system to segregate the people into credit score brackets to reduce the manual efforts.
## Task
Given a person’s credit-related information, build a machine learning model that can classify the credit score.

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
#from imblearn.over_sampling import SMOTE
import re

from sklearn.neighbors import LocalOutlierFactor
from sklearn.impute import SimpleImputer
from sklearn.ensemble import (BaggingClassifier,
                              ExtraTreesClassifier,
                              RandomForestClassifier, 
                              StackingClassifier,
                              HistGradientBoostingClassifier)
from xgboost import XGBClassifier
from sklearn.metrics import (classification_report,
                             accuracy_score, 
                             precision_score,
                             recall_score,
                             f1_score )

import pickle
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'xgboost'

In [2]:
data=pd.read_csv('C:\Users\Admin\Downloads\Credit Score Data/train.csv')
data.sample(10)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
42726,0x11058,CUS_0xafa3,July,Alwyn Scotts,26,484-09-6362,Engineer,28347.68,2091.306667,0,...,Good,747.72,36.739345,27 Years and 3 Months,No,0.000000,184.3605896855977,Low_spent_Small_value_payments,314.770076981069,Standard
7668,0x42ee,CUS_0x2d83,May,Abhijitn,18,307-73-4226,Lawyer,15325.73,1181.144167,6,...,Bad,4445.87,27.318986,11 Years and 7 Months,Yes,61.850784,57.35988952663905,High_spent_Small_value_payments,258.9037427251206,Standard
75509,0x1d06f,CUS_0xfd1,June,Frewy,32,389-55-6408,Architect,78666.56999999999,6485.547500,3,...,Standard,1498.7,31.796510,22 Years and 3 Months,No,247.851145,58.49699428673007,!@9#%8,582.2066110456252,Standard
84614,0x205c8,CUS_0x83ea,July,Lucia Mutikanif,34,869-20-6910,Media_Manager,14461.52,1010.126667,10,...,Bad,2566.1,28.311767,8 Years and 7 Months,NM,41.821719,70.5722075035085,Low_spent_Medium_value_payments,268.6187403359802,Standard
12974,0x6204,CUS_0x3592,July,Dominique Vidalonc,37,711-41-8889,Manager,34739.35,2909.945833,8,...,Standard,1408.63,35.143116,22 Years and 2 Months,NM,65.139247,171.94968127897778,Low_spent_Small_value_payments,343.905655549932,Standard
75704,0x1d196,CUS_0xaf49,January,Georgiopouloso,29,023-15-3883,Teacher,65804.04,NaN,0,...,Good,1450.47,34.748045,20 Years and 4 Months,No,76.328562,524.4673595707957,Low_spent_Large_value_payments,229.17107850458515,Good
74439,0x1ca29,CUS_0x453d,August,NaN,25,376-01-8404,Journalist,159327.84,13543.320000,3,...,Good,803.43,40.384657,17 Years and 9 Months,No,443.675781,198.3281768601753,High_spent_Large_value_payments,952.328042203774,Standard
909,0x1b53,CUS_0x7586,June,Stevee,32,790-90-6437,Engineer,55151.04,4704.920000,10,...,Bad,1743.83,39.269466,12 Years and 7 Months,Yes,171.823067,104.87753270580295,High_spent_Medium_value_payments,443.7913999132952,Poor
24822,0xa770,CUS_0x32d1,July,Ianq,39,173-69-9224,Journalist,58267.23,NaN,8,...,Standard,1896.36,28.922609,NaN,Yes,109.720351,89.35086809283303,High_spent_Large_value_payments,535.2890313215421,Poor
27046,0xb478,CUS_0xa138,July,Arefu,24,#F%$D@*&8,Developer,126664.28,10474.356667,7,...,Standard,270.36,44.028316,32 Years and 8 Months,Yes,149.180572,222.67655203429518,High_spent_Large_value_payments,915.578542800117,Poor


In [3]:
# Data Decription
data.describe(include='all')

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
count,100000,100000,100000,90015,100000,100000,100000,100000,84998.000000,100000.000000,...,100000,100000,100000.000000,90970,100000,100000.000000,95521,100000,98800,100000
unique,100000,12500,8,10139,1788,12501,16,18940,NaN,NaN,...,4,13178,NaN,404,3,NaN,91049,7,98792,3
top,0x1602,CUS_0xd40,January,Langep,38,#F%$D@*&8,_______,36585.12,NaN,NaN,...,Standard,1360.45,NaN,15 Years and 11 Months,Yes,NaN,__10000__,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard
freq,1,8,12500,44,2833,5572,7062,16,NaN,NaN,...,36479,24,NaN,446,52326,NaN,4305,25513,9,53174
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4194.170850,17.091280,...,NaN,NaN,32.285173,NaN,NaN,1403.118217,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3183.686167,117.404834,...,NaN,NaN,5.116875,NaN,NaN,8306.041270,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,303.645417,-1.000000,...,NaN,NaN,20.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1625.568229,3.000000,...,NaN,NaN,28.052567,NaN,NaN,30.306660,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3093.745000,6.000000,...,NaN,NaN,32.305784,NaN,NaN,69.249473,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5957.448333,7.000000,...,NaN,NaN,36.496663,NaN,NaN,161.224249,NaN,NaN,NaN,NaN


In [4]:
# Data Info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [5]:
# Checking for missing values
data.isnull().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

## Data Preprocessing

In [6]:
# cleaning Outstanding Debt
data['Outstanding_Debt']=data['Outstanding_Debt'].str.extract('(\d+)')
# cleaning Annual income
data['Annual_Income']=data['Annual_Income'].str.extract('(\d+)')
# cleaning changed credit limit
data['Changed_Credit_Limit']=data['Changed_Credit_Limit'].str.extract('(\d+)')
# Cleaning Number of loan
data['Num_of_Loan']=data['Num_of_Loan'].str.extract('(\d+)')
# Cleaning Age
data['Age']=data['Age'].str.extract('(\d+)')
# Cleaning Amount invested monthly
data['Amount_invested_monthly']=(data['Amount_invested_monthly'].astype('str')).str.extract('(\d+)')
# Cleaning Monthly Balance
data['Monthly_Balance']=(data['Monthly_Balance'].astype('str')).str.extract('(\d+)')
# Cleaning Number of Delayed Payment 
data['Num_of_Delayed_Payment']=data['Num_of_Delayed_Payment'].str.extract('(\d+)')
# Filling the None values with the median of Num of Delayed payment through simpleimputer
si=SimpleImputer(strategy='median')
si=si.fit_transform(data[['Num_of_Delayed_Payment']])
data[['Num_of_Delayed_Payment']]=si

In [7]:
# Credit_History_Age filtering
def History_age(age):
    try : 
        years = int("".join(re.findall('[0-9]',''.join(age.split("and")[0]))))
        month = int("".join(re.findall('[0-9]',''.join(age.split("and")[1]))))
        return years*12 + month
    except :
        return np.nan

In [8]:
data['Credit_History_Age'] = data['Credit_History_Age'].apply(History_age)

In [9]:
# filling empty value with most frequent value
si_type_of_loan=SimpleImputer(strategy='most_frequent')
si_type_of_loan=si_type_of_loan.fit(data[['Type_of_Loan']])
data[['Type_of_Loan']]=si_type_of_loan.transform(data[['Type_of_Loan']])

In [10]:
''' 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card',
     'Interest_Rate', 'Delay_from_due_date', 'Num_Credit_Inquiries',
       'Credit_Utilization_Ratio', 'Total_EMI_per_month','Monthly_Balance',
       'Amount_invested_monthly','Credit_History_Age','Changed_Credit_Limit '''

si=SimpleImputer()
si=si.fit(data[['Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card',
       'Interest_Rate', 'Delay_from_due_date', 'Num_Credit_Inquiries',
       'Credit_Utilization_Ratio', 'Total_EMI_per_month','Monthly_Balance','Amount_invested_monthly','Credit_History_Age','Changed_Credit_Limit']])
si=si.transform(data[['Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card',
       'Interest_Rate', 'Delay_from_due_date', 'Num_Credit_Inquiries',
       'Credit_Utilization_Ratio', 'Total_EMI_per_month','Monthly_Balance','Amount_invested_monthly','Credit_History_Age','Changed_Credit_Limit']])
data[['Monthly_Inhand_Salary','Num_Bank_Accounts', 'Num_Credit_Card',
       'Interest_Rate','Delay_from_due_date', 'Num_Credit_Inquiries',
       'Credit_Utilization_Ratio','Total_EMI_per_month','Monthly_Balance',
      'Amount_invested_monthly','Credit_History_Age','Changed_Credit_Limit']]=si

In [11]:
data.isnull().sum()

ID                             0
Customer_ID                    0
Month                          0
Name                        9985
Age                            0
SSN                            0
Occupation                     0
Annual_Income                  0
Monthly_Inhand_Salary          0
Num_Bank_Accounts              0
Num_Credit_Card                0
Interest_Rate                  0
Num_of_Loan                    0
Type_of_Loan                   0
Delay_from_due_date            0
Num_of_Delayed_Payment         0
Changed_Credit_Limit           0
Num_Credit_Inquiries           0
Credit_Mix                     0
Outstanding_Debt               0
Credit_Utilization_Ratio       0
Credit_History_Age             0
Payment_of_Min_Amount          0
Total_EMI_per_month            0
Amount_invested_monthly        0
Payment_Behaviour              0
Monthly_Balance                0
Credit_Score                   0
dtype: int64

In [12]:
# Deleting columns that are not relevant for the model
del data['ID']
del data['Name']
del data['Customer_ID']
del data['SSN']
del data['Month']

In [13]:
data.isnull().sum()

Age                         0
Occupation                  0
Annual_Income               0
Monthly_Inhand_Salary       0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Type_of_Loan                0
Delay_from_due_date         0
Num_of_Delayed_Payment      0
Changed_Credit_Limit        0
Num_Credit_Inquiries        0
Credit_Mix                  0
Outstanding_Debt            0
Credit_Utilization_Ratio    0
Credit_History_Age          0
Payment_of_Min_Amount       0
Total_EMI_per_month         0
Amount_invested_monthly     0
Payment_Behaviour           0
Monthly_Balance             0
Credit_Score                0
dtype: int64

In [14]:
data.select_dtypes(include='object').head()

,Age,Occupation,Annual_Income,Num_of_Loan,Type_of_Loan,Credit_Mix,Outstanding_Debt,Payment_of_Min_Amount,Payment_Behaviour,Credit_Score
0,23,Scientist,19114,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",_,809,No,High_spent_Small_value_payments,Good
1,23,Scientist,19114,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",Good,809,No,Low_spent_Large_value_payments,Good
2,500,Scientist,19114,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",Good,809,No,Low_spent_Medium_value_payments,Good
3,23,Scientist,19114,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",Good,809,No,Low_spent_Small_value_payments,Good
4,23,Scientist,19114,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",Good,809,No,High_spent_Medium_value_payments,Good


## OCCUPATION

In [15]:
print(data.select_dtypes(include='object')['Occupation'].value_counts())
fig=px.bar(data.select_dtypes(include='object')['Occupation'].value_counts())
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

_______          7062
Lawyer           6575
Architect        6355
Engineer         6350
Scientist        6299
Mechanic         6291
Accountant       6271
Developer        6235
Media_Manager    6232
Teacher          6215
Entrepreneur     6174
Doctor           6087
Journalist       6085
Manager          5973
Musician         5911
Writer           5885
Name: Occupation, dtype: int64


In [16]:
#cleaning Occupation
data['Occupation'].replace('_______','others',inplace=True)

In [17]:
# visualization after cleaning
print(data.select_dtypes(include='object')['Occupation'].value_counts())
fig=px.bar(data.select_dtypes(include='object')['Occupation'].value_counts())
fig.show()

others           7062
Lawyer           6575
Architect        6355
Engineer         6350
Scientist        6299
Mechanic         6291
Accountant       6271
Developer        6235
Media_Manager    6232
Teacher          6215
Entrepreneur     6174
Doctor           6087
Journalist       6085
Manager          5973
Musician         5911
Writer           5885
Name: Occupation, dtype: int64


## Credit Mix

In [18]:
print(data.select_dtypes(include='object')['Credit_Mix'].value_counts().sort_values())
fig=px.bar(data.select_dtypes(include='object')['Credit_Mix'].value_counts().sort_values())
fig.show()

Bad         18989
_           20195
Good        24337
Standard    36479
Name: Credit_Mix, dtype: int64


In [19]:
# Cleaning Credit Mix
# Good
good=data[['Credit_Mix','Credit_Score']][data['Credit_Score']=='Good']['Credit_Mix'].replace('_','Good')
# Standard
standard=data[['Credit_Mix','Credit_Score']][data['Credit_Score']=='Standard']['Credit_Mix'].replace('_','Standard')
# poor/Bad
bad=data[['Credit_Mix','Credit_Score']][data['Credit_Score']=='Poor']['Credit_Mix'].replace('_','Bad')
g_s_b=pd.concat([good,standard,bad])
g_s_b.sort_index(inplace=True)
data['Credit_Mix']=g_s_b

In [20]:
# Visualizationc After Cleaning Credit Mix
print(data.select_dtypes(include='object')['Credit_Mix'].value_counts().sort_values())
fig=px.bar(data.select_dtypes(include='object')['Credit_Mix'].value_counts().sort_values())
fig.show()

Bad         24858
Good        27959
Standard    47183
Name: Credit_Mix, dtype: int64


In [21]:
print(data['Payment_of_Min_Amount'].value_counts())
fig=px.bar(data['Payment_of_Min_Amount'].value_counts())
fig.show()

Yes    52326
No     35667
NM     12007
Name: Payment_of_Min_Amount, dtype: int64


In [22]:
# Cleaning payment of minimum amount
data['Payment_of_Min_Amount'].replace('NM','No',inplace=True)

In [23]:
# Visualization after cleaning
print(data['Payment_of_Min_Amount'].value_counts())
fig=px.bar(data['Payment_of_Min_Amount'].value_counts())
fig.show()

Yes    52326
No     47674
Name: Payment_of_Min_Amount, dtype: int64


In [24]:
print(data['Payment_Behaviour'].value_counts())
fig=px.bar(data['Payment_Behaviour'].value_counts())
fig.show()

Low_spent_Small_value_payments      25513
High_spent_Medium_value_payments    17540
Low_spent_Medium_value_payments     13861
High_spent_Large_value_payments     13721
High_spent_Small_value_payments     11340
Low_spent_Large_value_payments      10425
!@9#%8                               7600
Name: Payment_Behaviour, dtype: int64


In [25]:
# cleaning payment behaviour 
data['Payment_Behaviour'].replace('!@9#%8','other',inplace=True)

In [26]:
# visualization after cleaning
print(data['Payment_Behaviour'].value_counts())
fig=px.bar(data['Payment_Behaviour'].value_counts())
fig.show()

Low_spent_Small_value_payments      25513
High_spent_Medium_value_payments    17540
Low_spent_Medium_value_payments     13861
High_spent_Large_value_payments     13721
High_spent_Small_value_payments     11340
Low_spent_Large_value_payments      10425
other                                7600
Name: Payment_Behaviour, dtype: int64


In [27]:
print(data['Credit_Score'].value_counts())
fig=px.bar(data['Credit_Score'].value_counts())
fig.show()

Standard    53174
Poor        28998
Good        17828
Name: Credit_Score, dtype: int64


In [28]:
# Age visualization
fig=px.scatter(data['Age'].value_counts())
fig.show()

In [29]:
# taking care of outliers
lo_age=LocalOutlierFactor()
lo_age=lo_age.fit(data[['Age']])
data['Age']=pd.Series(lo_age.negative_outlier_factor_)

In [30]:
# visualization after taking care of outlier
fig=px.scatter(data['Age'].value_counts())
fig.show()

In [31]:
# Converting object to float
data[['Annual_Income','Num_of_Loan','Changed_Credit_Limit','Outstanding_Debt']]=data[['Annual_Income','Num_of_Loan','Changed_Credit_Limit','Outstanding_Debt']].astype('float64')

In [32]:
data.sample(5)

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
50432,-1.0,Developer,179193.0,14867.813333,5.0,3.0,8.0,1.0,Debt Consolidation Loan,1.0,...,Good,449.0,48.176599,255.000000,No,83.417292,233.000000,High_spent_Large_value_payments,1409.0,Good
77744,-1.0,Teacher,20756.0,4194.170850,8.0,10.0,16.0,7.0,"Credit-Builder Loan, Payday Loan, Debt Consoli...",52.0,...,Bad,4206.0,25.802700,221.195405,Yes,62.388472,243.000000,Low_spent_Small_value_payments,166.0,Poor
8088,-1.0,Entrepreneur,76113.0,6281.757500,7.0,6.0,16.0,0.0,Not Specified,15.0,...,Standard,1127.0,29.561537,323.000000,Yes,0.000000,305.000000,Low_spent_Large_value_payments,592.0,Standard
53363,-1.0,Manager,83265.0,4194.170850,7.0,8.0,30.0,6.0,"Student Loan, Credit-Builder Loan, Student Loa...",40.0,...,Bad,2224.0,31.197320,70.000000,Yes,296.609494,636.936286,Low_spent_Small_value_payments,333.0,Poor
46180,-1.0,Manager,72099.0,5842.350694,158.0,3.0,2.0,4.0,"Payday Loan, Personal Loan, Credit-Builder Loa...",15.0,...,Good,1421.0,36.878001,335.000000,No,426.797077,429.000000,Low_spent_Medium_value_payments,268.0,Good


In [33]:
# Cleaning type of loan
data['Type_of_Loan']=pd.DataFrame(data['Type_of_Loan'].value_counts().head(9).index[:])

In [34]:
data['Type_of_Loan']

0                  Not Specified
1            Credit-Builder Loan
2                  Personal Loan
3        Debt Consolidation Loan
4                   Student Loan
                  ...           
99995                        NaN
99996                        NaN
99997                        NaN
99998                        NaN
99999                        NaN
Name: Type_of_Loan, Length: 100000, dtype: object

In [35]:
si=SimpleImputer(strategy='most_frequent')
data['Type_of_Loan']=si.fit_transform(data[['Type_of_Loan']])

In [36]:
data['Type_of_Loan'].value_counts()

Auto Loan                  99992
Not Specified                  1
Credit-Builder Loan            1
Personal Loan                  1
Debt Consolidation Loan        1
Student Loan                   1
Payday Loan                    1
Mortgage Loan                  1
Home Equity Loan               1
Name: Type_of_Loan, dtype: int64

## Splitting the data into features and label

In [37]:
x,y=data.drop('Credit_Score',axis=1),data['Credit_Score']

In [38]:
x.select_dtypes(include=['object']).sample(5)

,Occupation,Type_of_Loan,Credit_Mix,Payment_of_Min_Amount,Payment_Behaviour
14922,Developer,Auto Loan,Standard,No,High_spent_Small_value_payments
94313,Manager,Auto Loan,Standard,Yes,Low_spent_Large_value_payments
35819,Teacher,Auto Loan,Good,No,Low_spent_Medium_value_payments
74468,Musician,Auto Loan,Bad,Yes,High_spent_Large_value_payments
2546,Writer,Auto Loan,Good,No,other


In [39]:
dummies=pd.get_dummies(x.select_dtypes(include=['object']))
dummies

,Occupation_Accountant,Occupation_Architect,Occupation_Developer,Occupation_Doctor,Occupation_Engineer,Occupation_Entrepreneur,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,...,Credit_Mix_Standard,Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes,Payment_Behaviour_High_spent_Large_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments,Payment_Behaviour_other
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,0,0,0,0,0,0,0,0,1,...,0,1,0,1,0,0,0,0,0,0
99996,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,0
99997,0,0,0,0,0,0,0,0,0,1,...,0,1,0,1,0,0,0,0,0,0
99998,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0


In [40]:
del x['Occupation']
del x['Type_of_Loan'] 
del x['Credit_Mix']
del x['Payment_of_Min_Amount']
del x['Payment_Behaviour']

In [41]:
x=x.join(dummies)

In [42]:
x.sample(5)

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Credit_Mix_Standard,Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes,Payment_Behaviour_High_spent_Large_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments,Payment_Behaviour_other
130,-1.0,92047.0,7591.59000,6.0,7.0,16.0,0.0,18.0,11.0,9.0,...,1,0,1,1,0,0,0,0,0,0
75669,-1.0,138206.0,4194.17085,1.0,2.0,8.0,3.0,14.0,5.0,4.0,...,1,1,0,0,1,0,0,0,0,0
85659,-1.0,42185.0,3495.43750,4.0,5.0,9.0,1.0,13.0,11.0,5.0,...,0,1,0,0,0,0,0,0,1,0
37162,-1.0,46410.0,3973.53500,8.0,6.0,17.0,9.0,50.0,19.0,18.0,...,0,0,1,0,0,0,0,0,1,0
61390,-1.0,79264.0,6863.40500,4.0,7.0,9.0,5.0,13.0,14.0,12.0,...,1,0,1,0,0,0,0,0,1,0


In [43]:
y=y.map({'Good':0,
      'Standard':1,
      'Poor':2})

In [44]:
y.value_counts()

1    53174
2    28998
0    17828
Name: Credit_Score, dtype: int64

In [45]:
smote=SMOTE()
x_smote,y_smote=smote.fit_resample(x,y)

In [46]:
y_smote.value_counts(normalize=True)

0    0.333333
1    0.333333
2    0.333333
Name: Credit_Score, dtype: float64

In [47]:
x_train,x_test,y_train,y_test=train_test_split(x_smote,y_smote,test_size=0.2,random_state=42)

In [48]:
scalar = PowerTransformer(method='yeo-johnson', standardize=True).fit(x_train)

In [106]:
def credit_function(y_pred,y_test):
    accuracy=accuracy_score(y_pred,y_test)
    recall=recall_score(y_test,y_pred,average='macro')
    precision=precision_score(y_pred,y_test,average='macro')
    f1=f1_score(y_pred,y_test,average='macro')
    
    print(f'Accuracy Score: {accuracy*100:.2f}%')
    print(f'Recall Score: {recall*100:.2f}%')
    print(f'Precision Score: {precision*100:.2f}%')
    print(f'f1 Score Score: {f1*100:.2f}%')    

## Model Evaluation

In [50]:
bagging = BaggingClassifier(n_jobs=-1)
extraTrees = ExtraTreesClassifier(max_depth=10, n_jobs=-1)
randomForest = RandomForestClassifier(n_jobs=-1)
histGradientBoosting = HistGradientBoostingClassifier()
XGB = XGBClassifier(n_jobs=-1)

model = StackingClassifier([
    ('bagging', bagging),
    ('extraTress', extraTrees),
    ('randomforest', randomForest),
    ('histGradientBoosting', histGradientBoosting),
    ('XGB', XGB)
    ], n_jobs=-1)
    
model=model.fit(x_train, y_train)
y_pred=model.predict(x_test)

In [105]:
credit_function(y_pred,y_test)

Accuracy Score: 86.64%
Recall Score: 86.63%
Precision Score: 86.63%
f1 Score Score: 86.59%


In [99]:
print(f'Classification report {classification_report(y_pred,y_test)}')

Classification report               precision    recall  f1-score   support

           0       0.92      0.89      0.90     11063
           1       0.81      0.84      0.82     10272
           2       0.87      0.87      0.87     10570

    accuracy                           0.87     31905
   macro avg       0.87      0.87      0.87     31905
weighted avg       0.87      0.87      0.87     31905



## Save Model

In [108]:
pickle.dump(model,open('stackingclassifier.pkl','wb'))